# Task IV - Time Series Forecasting

In [ ]:
import numpy as np

In [2]:
import pandas as pd

#Reading the data
X_KV = pd.read_csv('../X_KV.csv')   #JNSM_KV_flashcrowd_1 
Y_KV = pd.read_csv('../Y_KV.csv')   
X_VoD = pd.read_csv('../X_VoD.csv') #JNSM_VoD_flashcrowd_1 
Y_VoD = pd.read_csv('../Y_VoD.csv') 

X_KV_2 = pd.read_csv('../X_KV_2.csv')    #JNSM_KV_flashcrowd_2
Y_KV_2 = pd.read_csv('../Y_KV_2.csv')    
X_VoD_2 = pd.read_csv('../X_VoD_2.csv')  #JNSM_VoD_flashcrowd_2
Y_VoD_2 = pd.read_csv('../Y_VoD_2.csv')     

#Concatenate datasets
X_KV = pd.concat([X_KV, X_KV_2], ignore_index=True)
Y_KV = pd.concat([Y_KV, Y_KV_2], ignore_index=True)
X_VoD = pd.concat([X_VoD, X_VoD_2], ignore_index=True)
Y_VoD = pd.concat([Y_VoD, Y_VoD_2], ignore_index=True)

#Remove the first two columns that index the samples and retrieve all other values by using iloc()
X_KV = X_KV.iloc[:,2:]  
Y_KV = Y_KV.iloc[:, 2:] 
X_VoD = X_VoD.iloc[:,2:]
Y_VoD = Y_VoD.iloc[:, 2:]

#FedCSIS Data
X_CSIS = pd.read_csv('../FedCSIS_X.csv')   
Y_CSIS = pd.read_csv('../FedCSIS_Y.csv')

X_CSIS = X_CSIS.iloc[:,1:] 
Y_CSIS = Y_CSIS.iloc[:,1:]

In [3]:
#Retrieving Training and Test data
%store -r Y_KV_train Y_KV_test
%store -r Y_VoD_train Y_VoD_test
%store -r Y_CSIS_train Y_CSIS_test

Y_KV_train = Y_KV_train.values
Y_KV_test = Y_KV_test.values
Y_VoD_train = Y_VoD_train.values
Y_VoD_test = Y_VoD_test.values
Y_CSIS_train = Y_CSIS_train.values
Y_CSIS_test = Y_CSIS_test.values

### Task IV.1 - Autoregression
Fit an Autoregression (AR) model to the KTH time series. Perform forecasting using the AR model, which formulates the next step in the sequence as a linear function of the observations at previous time steps. The method is suitable for time series without trend and seasonal components. Evaluate the method for the AR model parameter p = 1, ..., 10.

In [16]:
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import mean_absolute_error

#Function that computes the next predition given the parameters and the past values of the time series
def f(params, past, lag):
    prediction = params[0]
    for i in range(0,lag):
        prediction += params[i+1]*past[lag-1-i]
    return prediction

#Rollling Prediction given different lag values and fixed h=10
def AR_prediction(lag,s,training_set,test_set):
    
    Y = np.concatenate((training_set, test_set))
    model = AutoReg(training_set, lags=lag).fit()
    params = model.params
    
    matrix = np.empty((len(test_set)-10*s+1,10))
    
    if s>1: #First row when s>1
        past_values = Y[len(training_set)-s*lag: len(training_set): s]
        #matrix[0] = f(params, past_values, lag)
        matrix[0][0] = f(params, past_values, lag)
        for j in range(1,10):
            past_values = np.delete(past_values,0,0)
            past_values = np.append(past_values,matrix[0][j-1])
            matrix[0][j] = f(params, past_values, lag)
            
    if s==1: #First row when s=1
        matrix[0] = model.predict(start=len(training_set), end=len(training_set)+10-1)
    
    for t in range(1,len(test_set)-10*s+1): #Remaining rows
        past_values = Y[len(training_set)-s*lag + t: len(training_set) + t: s]
        matrix[t][0] = f(params, past_values, lag)
        
        for j in range(1,10):
            past_values = np.delete(past_values,0,0)
            past_values = np.append(past_values,matrix[t][j-1])
            matrix[t][j] = f(params, past_values, lag)
    
    return matrix

#Function to compute the actual test values arranged by horizon value
def Y_new(targets, l, h, s):
    targets = np.array(targets)
        
    if (l in range (1,11)) and (h in range(1,11)):
        matrix = np.empty((targets.shape[0]-s*h+1,h))   
        for i in range(0, targets.shape[0]-s*h+1):
            matrix[i] = np.concatenate([[targets[j]] for j in range(i,i+s*h,s)])
        return matrix
    else:  
        print('Lag and horizon need to take values 1,2,...,10')

#Compute the error between the actual test values and the predicted ones
def errors(training_set, test_set, h,s):
    if h in range(1,11):
        errors = np.empty(10)
        for l in range(1,11):
            errors[l-1] = (1/np.mean(Y_new(test_set,l,10,s)[:,:h]))*mean_absolute_error(AR_prediction(l,s,training_set,test_set)[:,:h],Y_new(test_set,l,10,s)[:,:h])
        return errors
    else: 
        print('Horizon value needs to be 0,...,10')

In [23]:
#Matrices for each service that display the NMAE given h = 1,...,10 (rows) and l = 1,...,10 (columns)

error_matrix_KV_AR = []
error_matrix_VoD_AR = []
error_matrix_CSIS_AR = []
for h in range(1,11):
    error_matrix_KV_AR.append(errors(Y_KV_train,Y_KV_test,h,1)) 
    error_matrix_VoD_AR.append(errors(Y_VoD_train,Y_VoD_test,h,30))
    error_matrix_CSIS_AR.append(errors(Y_CSIS_train,Y_CSIS_test,h,1))
    
%store error_matrix_KV_AR error_matrix_VoD_AR error_matrix_CSIS_AR

Stored 'error_matrix_KV_AR' (list)
Stored 'error_matrix_VoD_AR' (list)
Stored 'error_matrix_CSIS_AR' (list)


In [651]:
#Obtain Latex code 

import tabulate
from tabulate import tabulate

np.set_printoptions(precision=4)
error_matrix_KV_AR_new = np.array(error_matrix_KV_AR) * 100000
error_matrix_VoD_AR_new = np.array(error_matrix_VoD_AR) * 10000
error_matrix_CSIS_AR_new = np.array(error_matrix_CSIS_AR) * 100000

print(tabulate(error_matrix_KV_AR_new, tablefmt="latex", floatfmt=".0f"))
print(tabulate(error_matrix_VoD_AR_new, tablefmt="latex", floatfmt=".0f"))
print(tabulate(error_matrix_CSIS_AR_new, tablefmt="latex", floatfmt=".0f"))

\begin{tabular}{rrrrrrrrrr}
\hline
 2389 & 2239 & 2188 & 2172 & 2165 & 2159 & 2150 & 2140 & 2139 & 2137 \\
 2554 & 2350 & 2287 & 2265 & 2253 & 2240 & 2227 & 2214 & 2210 & 2206 \\
 2712 & 2446 & 2373 & 2344 & 2327 & 2308 & 2289 & 2273 & 2267 & 2261 \\
 2861 & 2542 & 2454 & 2418 & 2394 & 2368 & 2345 & 2324 & 2315 & 2308 \\
 2994 & 2633 & 2529 & 2486 & 2456 & 2424 & 2395 & 2370 & 2358 & 2350 \\
 3108 & 2717 & 2597 & 2547 & 2511 & 2473 & 2439 & 2410 & 2395 & 2385 \\
 3207 & 2796 & 2662 & 2604 & 2563 & 2519 & 2480 & 2446 & 2428 & 2415 \\
 3292 & 2868 & 2723 & 2659 & 2612 & 2563 & 2520 & 2481 & 2459 & 2444 \\
 3364 & 2936 & 2781 & 2712 & 2660 & 2607 & 2558 & 2515 & 2489 & 2472 \\
 3427 & 3000 & 2836 & 2762 & 2707 & 2648 & 2596 & 2548 & 2519 & 2500 \\
\hline
\end{tabular}
\begin{tabular}{rrrrrrrrrr}
\hline
 1506 & 1503 & 1508 & 1508 & 1508 & 1508 & 1508 & 1509 & 1511 & 1516 \\
 1478 & 1478 & 1483 & 1482 & 1483 & 1482 & 1483 & 1483 & 1486 & 1491 \\
 1538 & 1538 & 1542 & 1542 & 1542 & 1542 & 15

### Task IV.2 - Moving Average
Fit a Moving Average (MA) model to the KTH time series. Perform forecasting using the MA model, which formulates the next step in the sequence as a linear function of the residual errors from a mean process at previous time steps. Note that MA is different from calculating the moving average of a time series. The method is suitable for time series without trend and seasonal components. Evaluate the method for the model parameter q = 1, ..., 10.

In [390]:
from statsmodels.tsa.arima.model import ARIMA

#Function that computes the next predition given the parameters and the past residuals of the time series
def f(param, resid, lag):
    prediction = param[0]
    for i in range(0,lag):
        prediction += param[i+1]*resid[lag-1-i]
    return prediction

#Compute Residuals on the test set
def residuals(lag,training_set,test_set):
    
    model = ARIMA(endog=training_set, order=(0, 0, lag)).fit()
    param = model.params[:lag+1]
    
    resid = model.resid
    #past_resid = resid[len(training_set)-lag: len(training_set)]
    
    for i in range(0,len(test_set)):
        
        prediction = f(param, resid[len(resid)-lag:], lag)
        resid_new = prediction - test_set[i]
        resid = np.append(resid,resid_new)
        
        #past_resid = np.delete(past_resid,0,0)
        #past_resid = np.append(past_resid,resid_new)
    
    return resid
 
    
#Rollling Prediction given different lag values and fixed h=10
def prediction_MA(lag,s,training_set,test_set):
    
    #In case residuals function doesnt work
    #data = np.concatenate((training_set,test_set))
    #model = ARIMA(endog=data, order=(0, 0, lag)).fit()
    #param = model.params[:lag+1]
    #resid = model.resid
    
    model = ARIMA(endog=training_set, order=(0, 0, lag)).fit()
    param = model.params[:lag+1]
    resid = residuals(lag,training_set,test_set)
    
    matrix = np.empty((len(test_set)-10*s+1,10))
    
    for i in range(0,len(test_set)-s*10+1):
        for j in range(0,10):
            laged_resid = resid[len(training_set) - s*lag + i+j: len(training_set)+i+j: s]
            matrix[i][j] = f(param, laged_resid, lag)
    return matrix


#Function to compute the actual test values arranged by horizon value
def Y_new(targets, l, h, s):
    targets = np.array(targets)
        
    if (l in range (1,11)) and (h in range(1,11)):
        matrix = np.empty((targets.shape[0]-s*h+1,h))   
        for i in range(0, targets.shape[0]-s*h+1):
            matrix[i] = np.concatenate([[targets[j]] for j in range(i,i+s*h,s)])
        return matrix
    else:  
        print('Lag and horizon need to take values 1,2,...,10')
        
        
#Compute the error between the actual test values and the predicted ones
def errors(training_set, test_set, h,s):
    if h in range(1,11):
        errors = np.empty(10)
        for l in range(1,11):
            errors[l-1] = (1/np.mean(Y_new(test_set,l,10,s)[:,:h]))*mean_absolute_error(prediction_MA(l,s,training_set,test_set)[:,:h],Y_new(test_set,l,10,s)[:,:h])
        return errors
    else: 
        print('Horizon value needs to be 0,...,10')

In [ ]:
#Matrices for each service that display the NMAE given h = 0,1,...,10 (rows) and l = 0,1,...,10 (columns)

error_matrix_KV_MA = []
error_matrix_VoD_MA = []
error_matrix_CSIS_MA = []
for h in range(1,11):
    error_matrix_KV_MA.append(errors(Y_KV_train,Y_KV_test,h,1)) 
    error_matrix_VoD_MA.append(errors(Y_VoD_train,Y_VoD_test,h,30))
    error_matrix_CSIS_MA.append(errors(Y_CSIS_train,Y_CSIS_test,h,1))
    
%store error_matrix_CSIS_MA

In [655]:
#Obtain Latex code 

import tabulate
from tabulate import tabulate

np.set_printoptions(precision=4)
error_matrix_KV_MA_new = np.array(error_matrix_KV_MA) * 100000
error_matrix_VoD_MA_new = np.array(error_matrix_VoD_MA) * 10000
error_matrix_CSIS_MA_new = np.array(error_matrix_CSIS_MA) * 100000

print(tabulate(error_matrix_KV_MA_new, tablefmt="latex", floatfmt=".0f"))
print(tabulate(error_matrix_VoD_MA_new, tablefmt="latex", floatfmt=".0f"))
print(tabulate(error_matrix_CSIS_MA_new, tablefmt="latex", floatfmt=".0f"))

\begin{tabular}{rrrrrrrrrr}
\hline
 3126 & 2773 & 2610 & 2492 & 2412 & 2379 & 2351 & 2319 & 2295 & 2276 \\
 3126 & 2773 & 2609 & 2492 & 2412 & 2378 & 2350 & 2319 & 2295 & 2275 \\
 3126 & 2773 & 2609 & 2491 & 2412 & 2378 & 2350 & 2319 & 2295 & 2275 \\
 3126 & 2773 & 2610 & 2492 & 2412 & 2378 & 2350 & 2319 & 2295 & 2276 \\
 3126 & 2773 & 2610 & 2492 & 2412 & 2378 & 2350 & 2319 & 2295 & 2276 \\
 3127 & 2774 & 2610 & 2492 & 2412 & 2378 & 2350 & 2319 & 2296 & 2276 \\
 3127 & 2774 & 2610 & 2492 & 2412 & 2379 & 2351 & 2319 & 2296 & 2276 \\
 3127 & 2774 & 2610 & 2492 & 2412 & 2379 & 2351 & 2319 & 2296 & 2276 \\
 3127 & 2774 & 2610 & 2492 & 2412 & 2379 & 2351 & 2319 & 2296 & 2276 \\
 3127 & 2774 & 2610 & 2492 & 2412 & 2379 & 2351 & 2319 & 2296 & 2276 \\
\hline
\end{tabular}
\begin{tabular}{rrrrrrrrrr}
\hline
 1551 & 1561 & 1612 & 1664 & 1698 & 1722 & 1743 & 1764 & 1780 & 1797 \\
 1546 & 1589 & 1650 & 1701 & 1732 & 1753 & 1772 & 1792 & 1810 & 1826 \\
 1584 & 1636 & 1692 & 1736 & 1761 & 1777 & 17

### Task IV.3 - Autoregressive Integrated Moving Average
Fit an Autoregressive Integrated Moving Average (ARIMA) model to the KTH time series. Perform forecasting using the ARIMA model, which formulates the next step in the sequence as a linear func- tion of the differenced observations and residual errors at previous time steps. The ARIMA method combines both AR and MA including a differencing pre-processing step of the sequence to make the sequence stationary, called integration (I). ARIMA is suitable for time series with trends and with- out seasonal components. Evaluate the method for model parameters (p,d,q) as follows: d = 1; p = 1,...,10,q = 1,...,5.

In [678]:
from statsmodels.tsa.arima.model import ARIMA

#Function that computes the next predition given the parameters and the past residuals of the time series
def f(param, resid, past, lag,q):
    prediction = 0
    for i in range(0,lag):
        prediction += param[i]*past[lag-1-i]
    for i in range(0,q):
        prediction += param[i+lag]*resid[q-1-i]
    return prediction


#Compute Residuals on the test set
def residuals(lag,q,training_set,test_set):
    
    data = np.concatenate((training_set,test_set))
    model = ARIMA(endog=data, order=(q, 1, lag)).fit()
    param = model.params[:lag+q]
    resid = model.resid
    
    return resid
    

#Rollling Prediction given different lag values and fixed h=10
def prediction_ARIMA(lag,q,s,training_set,test_set):
    
    model = ARIMA(endog=training_set, order=(q, 1, lag)).fit()
    param = model.params[:lag+q]
    resid = residuals(lag,q,training_set,test_set)
    
    data = np.concatenate((training_set,test_set))
    diff = np.diff(data,n=1)
    past_diff = diff[len(training_set)-s*lag+s-2:len(training_set)-1:s]
    past_resid = resid[len(training_set)-s*q+s-1:len(training_set):s]
    
    matrix = np.empty((len(test_set)-10*s+1,10))
    
    if s>1: #First row when s>1
        diff_new = np.diff(training_set,n=1)
        matrix[0][0] = f(param, past_resid, past_diff, lag,q)+training_set[len(training_set)-1]
        for j in range(1,10):
            diff_new = np.append(diff_new,f(param, past_resid, past_diff, lag,q))
            past_diff = diff_new[len(training_set)-s*lag+s-2+j:len(training_set)-1+j:s]
            past_resid = resid[len(training_set)-s*q+s-1+j:len(training_set)+j:s]
            matrix[0][j] = f(param, past_resid, past_diff, lag,q)
            
    if s==1: #First row when s=1
        matrix[0] = model.predict(start=len(training_set), end=len(training_set)+10-1)
    
    
    for t in range(1,len(test_set)-10*s+1): #Remaining rows
        past_diff = diff[len(training_set)-s*lag+s-2 + t: len(training_set)-1 + t: s]
        past_resid = resid[len(training_set)-s*q+s-1 + t:len(training_set)+t:s]
        
        matrix[t][0] = f(param, past_resid, past_diff, lag,q)+data[len(training_set)-1+t]
        
        #diff_new = np.diff(data[:len(training_set)-1+t+1],n=1)
        diff_new = diff[:len(training_set)-1+t]
        for j in range(1,10):
            diff_new = np.append(diff_new,f(param, past_resid, past_diff, lag,q))
            past_diff = diff_new[len(training_set)-s*lag+s-2+j+t:len(training_set)-1+j+t:s]
            past_resid = resid[len(training_set)-s*q+s-1+j+t:len(training_set)+j+t:s]
            
            matrix[t][j] = f(param, past_resid, past_diff, lag,q) + data[len(training_set)-1+t+j]
    
    return matrix


#Function to compute the actual test values arranged by horizon value
def Y_new(targets, l, h, s):
    targets = np.array(targets)
        
    if (l in range (1,11)) and (h in range(1,11)):
        matrix = np.empty((targets.shape[0]-s*h+1,h))   
        for i in range(0, targets.shape[0]-s*h+1):
            matrix[i] = np.concatenate([[targets[j]] for j in range(i,i+s*h,s)])
        return matrix
    else:  
        print('Lag and horizon need to take values 1,2,...,10')
        
        
#Compute the error between the actual test values and the predicted ones
def errors(training_set, test_set, h,s):
    if h in range(1,11):
        errors = np.empty((5,10))
        for lag in range(1,11):
            for q in range(1,6):
                errors[q-1][lag-1] = (1/np.mean(Y_new(test_set,lag,10,s)[:,:h]))*mean_absolute_error(prediction_ARIMA(lag,q,s,training_set,test_set)[:,:h],Y_new(test_set,lag,10,s)[:,:h])
        return errors
    else: 
        print('Horizon value needs to be 0,...,10')

In [ ]:
error_matrix_KV_ARIMA = np.empty((5,10))
error_matrix_KV_ARIMA = errors(Y_KV_train, Y_KV_test, 5,1)
%store error_matrix_KV_ARIMA

error_matrix_CSIS_ARIMA = np.empty((5,10))
error_matrix_CSIS_ARIMA = errors(Y_CSIS_train, Y_CSIS_test, 5,1)
%store error_matrix_CSIS_ARIMA 

error_matrix_VoD_ARIMA = np.empty((5,10))
error_matrix_VoD_ARIMA = errors(Y_VoD_train, Y_VoD_test, 5,30)
%store error_matrix_VoD_ARIMA 

In [685]:
#Obtain Latex code 

import tabulate
from tabulate import tabulate

np.set_printoptions(precision=4)
error_matrix_KV_ARIMA_new = np.array(error_matrix_KV_ARIMA) * 100000
error_matrix_CSIS_ARIMA_new = np.array(error_matrix_CSIS_ARIMA) * 100000

print(tabulate(error_matrix_KV_ARIMA_new, tablefmt="latex", floatfmt=".0f"))
print(tabulate(error_matrix_CSIS_ARIMA_new, tablefmt="latex", floatfmt=".0f"))

\begin{tabular}{rrrrrrrrrr}
\hline
 2170 & 5407 &  4666 &  4154 & 3600 & 3017 &  2508 &  3854 & 2894 & 2512 \\
 3490 & 2321 & 15358 &  9770 & 8990 & 3526 & 10827 & 11780 & 9031 & 5676 \\
 3565 & 3017 &  3233 & 18716 & 5115 & 3664 &  4328 &  4892 & 5066 & 3677 \\
 3546 & 3130 &  4498 &  2358 & 3029 & 3498 &  3338 &  5489 & 3905 & 4103 \\
 3561 & 5845 &  3217 &  5577 & 2420 & 4138 &  5647 &  5131 & 3979 & 4495 \\
\hline
\end{tabular}
\begin{tabular}{rrrrrrrrrr}
\hline
 2269 & 2341 & 2609 & 2752 & 2744 & 2641 & 3016 & 2362 & 2438 & 2344 \\
 2364 & 2279 & 3731 & 3044 & 3267 & 3411 & 3322 & 3063 & 2888 & 3078 \\
 2410 & 2392 & 2290 & 2829 & 3508 & 4217 & 3947 & 3355 & 3195 & 3392 \\
 2402 & 2423 & 4620 & 3246 & 2355 & 4071 & 3745 & 3396 & 3251 & 3488 \\
 2353 & 2422 & 2448 & 3992 & 2287 & 2359 & 4373 & 3398 & 3531 & 3621 \\
\hline
\end{tabular}


### Task IV.4 - Exponential Smoothing
Perform forecasting using the exponential smoothing method as a baseline method. This method uses a weighted average of l = 10 previous values (y(t−l),...,y(t)) to predict the expected outcome at the next time step (t+1). The smoothing constant of the method α = 0.5.

In [301]:
#Function that computes the next predition given the parameters and the past values of the time series
def f(past, lag):
    params = []
    for i in range(1,lag+1):
        params.append(0.5**i)
    params = np.array(params)
    
    prediction = 0
    for i in range(0,lag):
        prediction += params[i]*past[lag-1-i]
    return prediction


#Rollling Prediction given different lag values and fixed h=10
def ES_prediction(lag,s,training_set,test_set):
    
    Y = np.concatenate((training_set, test_set))
    
    params = []
    for i in range(1,lag+1):
        params.append(0.5**i)
    params = np.array(params)
    
    matrix = np.empty((len(test_set)-10*s+1,10))
    
    past_values = Y[len(training_set)-s*lag: len(training_set): s]
    matrix[0][0] = f(past_values, lag)
    for j in range(1,10):
        past_values = np.delete(past_values,0,0)
        past_values = np.append(past_values,matrix[0][j-1])
        matrix[0][j] = f(past_values, lag)
    
    for t in range(1,len(test_set)-10*s+1): #Remaining rows
        past_values = Y[len(training_set)-s*lag + t: len(training_set) + t: s]
        matrix[t][0] = f(past_values, lag)
        
        for j in range(1,10):
            past_values = np.delete(past_values,0,0)
            past_values = np.append(past_values,matrix[t][j-1])
            matrix[t][j] = f(past_values, lag)
    
    return matrix

#Function to compute the actual test values arranged by horizon value
def Y_new(targets, l, h, s):
    targets = np.array(targets)
        
    if (l in range (1,11)) and (h in range(1,11)):
        matrix = np.empty((targets.shape[0]-s*h+1,h))   
        for i in range(0, targets.shape[0]-s*h+1):
            matrix[i] = np.concatenate([[targets[j]] for j in range(i,i+s*h,s)])
        return matrix
    else:  
        print('Lag and horizon need to take values 1,2,...,10')

#Compute the error between the actual test values and the predicted ones
def errors(training_set, test_set, h,s):
    if h in range(1,11):
        errors = np.empty(10)
        for l in range(1,11):
            errors[l-1] = (1/np.mean(Y_new(test_set,l,10,s)[:,:h]))*mean_absolute_error(ES_prediction(l,s,training_set,test_set)[:,:h],Y_new(test_set,l,10,s)[:,:h])
        return errors
    else: 
        print('Horizon value needs to be 0,...,10')

In [304]:
#Matrices for each service that display the NMAE given h = 0,1,...,10 (rows) and l = 0,1,...,10 (columns)

errors_KV_ES = []
errors_VoD_ES = []
errors_CSIS_ES = []
for h in range(1,11):
    errors_KV_ES.append(errors(Y_KV_train,Y_KV_test,h,1)) 
    errors_VoD_ES.append(errors(Y_VoD_train,Y_VoD_test,h,30))
    errors_CSIS_ES.append(errors(Y_CSIS_train,Y_CSIS_test,h,1))

%store errors_KV_ES errors_VoD_ES errors_CSIS_ES

Stored 'errors_KV_ES' (list)
Stored 'errors_VoD_ES' (list)
Stored 'errors_CSIS_ES' (list)


In [659]:
#Obtain Latex code 

import tabulate
from tabulate import tabulate

np.set_printoptions(precision=4)
error_matrix_KV_ES_new = np.array(errors_KV_ES) * 10000
error_matrix_VoD_ES_new = np.array(errors_VoD_ES) * 10000
error_matrix_CSIS_ES_new = np.array(errors_CSIS_ES) * 10000

print(tabulate(error_matrix_KV_ES_new, tablefmt="latex", floatfmt=".0f"))
print(tabulate(error_matrix_VoD_ES_new, tablefmt="latex", floatfmt=".0f"))
print(tabulate(error_matrix_CSIS_ES_new, tablefmt="latex", floatfmt=".0f"))

\begin{tabular}{rrrrrrrrrr}
\hline
 5000 & 2500 & 1250 &  634 &  352 & 249 & 223 & 218 & 217 & 218 \\
 6250 & 3125 & 1563 &  788 &  424 & 279 & 236 & 227 & 226 & 227 \\
 7083 & 3750 & 1875 &  943 &  498 & 310 & 250 & 236 & 234 & 234 \\
 7656 & 4297 & 2188 & 1098 &  572 & 343 & 264 & 243 & 240 & 240 \\
 8063 & 4781 & 2485 & 1254 &  648 & 378 & 278 & 251 & 245 & 245 \\
 8359 & 5209 & 2767 & 1406 &  723 & 413 & 293 & 257 & 250 & 249 \\
 8583 & 5586 & 3036 & 1556 &  799 & 448 & 308 & 264 & 254 & 252 \\
 8755 & 5921 & 3292 & 1702 &  874 & 483 & 323 & 270 & 257 & 255 \\
 8891 & 6218 & 3534 & 1845 &  948 & 519 & 338 & 277 & 261 & 258 \\
 9001 & 6482 & 3765 & 1985 & 1022 & 555 & 354 & 283 & 265 & 261 \\
\hline
\end{tabular}
\begin{tabular}{rrrrrrrrrr}
\hline
 5067 & 3071 & 2249 & 1864 & 1677 & 1585 & 1540 & 1517 & 1506 & 1500 \\
 6291 & 3546 & 2470 & 1985 & 1750 & 1635 & 1579 & 1551 & 1537 & 1530 \\
 7112 & 4056 & 2729 & 2145 & 1863 & 1725 & 1657 & 1624 & 1607 & 1599 \\
 7678 & 4537 & 2954 & 2